# Домашнее задание к лекции "Регулярные выражения"
Для зачета необходимо выполнить 4 задания.  
Для зачета с отличием – все.

## Задание 1

Напишите функцию, которая принимает на вход строку и проверяет является ли она валидным транспортным номером. Если да, то функция должна возвращать отдельно номер и регион.

In [1]:
import random
import re

validLetters = "АВЕКМНОРСТУХ"

def randAutoLetter(onlyValidLetters = False):
    '''Возвращает 1 букву русского алфавита для автознака (корректную или любую - в зависимости от onlyValidLetters)'''
    if onlyValidLetters:
        return random.choice(validLetters)
    else:
        allCyrrilicLetters = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
        return  random.choice(allCyrrilicLetters)

def randAutoNum(onlyValidLetters = False):
    '''Генерирует случайный номер автомобильного номера.
    В зависимости от onlyValidLetters - номер либо будет сгенерирован либо правильным, 
    либо - с использованием любых русских букв (т.е вероятно некорректным).'''
    randNumber1 = random.randrange(0,999)
    randNumber2 = random.randrange(0,125)
    
    autoNum = f"{randAutoLetter(onlyValidLetters)}{randNumber1:03}{randAutoLetter(onlyValidLetters) * 2}{randNumber2:02}"
    return autoNum

def validateAutoNum(autoNum):
    possibleLetterGroup = "[" + validLetters + "]" 
    pattern = r"(" + possibleLetterGroup + "\d{3}" + possibleLetterGroup + "{2})(\d{1,3})"
    result = re.findall(pattern, autoNum.upper())

    if len(result) == 0:           # Нет совпадения
        return 
    else:
        if int(result[0][1]) <= 0: # Регион <= 0
            return 
    return result

for r in range(10):
    randNum = randAutoNum(random.randint(0,1))
    print(randNum, validateAutoNum(randNum))

## Задание 2

Напишите функцию, которая проверяет e-mail адрес на валидность по стандарту RFC822: http://www.w3.org/Protocols/rfc822/. Для этого постарайтесь написать (в крайнем случае найти) самый полный шаблон для проверки e-mail адресов через регулярное выражение. Разберитесь и напишите примеры, какие ваш шаблон может не охватить.

In [2]:
def validateEmailAddress(address):
    #pattern = r"((:?[\"'0-9A-z\+\.-]|([\"'])+?[\s\S]+?\3)+@[\d\w\.-]+\.(:?\w)*)"           # Вариант 1
    pattern = r"^[0-9\w\.\%\/\-\+\*\!]+@(:?[0-9A-z\-]|[0-9A-z\-]+\.?[0-9A-z\-]+)+\.{1}\w*$" # Вариант 2
    
    # http://emailregex.com/
    #pattern = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"                        # Этот работает получше)
    
    result = re.match(pattern,address)
    if result is None:
        return False
    else:
        return True

emails = ([
            "n.john.smith@gmail.com", "87victory@hotmail.com", "!#mary-=@msca.net","abc.@insta.com",
            "abc..de@insta.com","abc-@insta.com", "abc-def@insta.com","abc+@insta.com",
            "abc123..45@insta.com", "abc%@insta.com","abc\\@insta.com", "abc/@insta.com",
            "abc@insta-.com", "abc@insta123.com","abc@inta.co.in", "abc@@insta.com.com.com",
            "abc@insta..com","abc@insta%.com", "abc@insta/.com","abc@insta\\.com",
            "abc1.1@domain.com","abc.abc@domain.com", "abc_41.15768@domain.com",
            "12345@domain.com","12.12.23@domain.com", "ABC.ABC.ABC.ABC0046@domain.com",
            "abc_123_4595@domain.com", "abc@asdas.co.in"
        ])

for email in emails:
    if validateEmailAddress(email):
        print(f'The email {email} is a valid email')
    else:
        print(f'The email {email} is invalid')  

## Задание 3
При помощи регулярных выражений напишите функцию, которая будет возвращать акроним переданного в нее текста.

In [3]:
def acronym(text, sep="."):
    pattern = "(:?(\w)\w*\n*){1,}"
    patternMatch = re.findall(pattern, text)
    return [x[1].upper()+sep for x in patternMatch]
    
text = ([
    "Жаркое дневное Египетское марево.",
    "Летом Египет кажется целиком искусственным, и ",
    "при относительно",
    "обширной орошаемой площади",
    'недостаточно аэрированным.',
    "При абстиненции йода требуется орошать низины ежедневно."
    ])

print(" ".join(["".join(acronym(txt,"")) for txt in text]))

ЖДЕМ ЛЕКЦИИ ПО ООП НА ПАЙТОНЕ


## Задание 4
Напишите функцию, которая будет удалять все последовательные повторы слов из заданной строки.

In [4]:
def removeDublicates(text):
    return re.sub(r"(\w+ )\1{2,}", r"\1", text)
    
removeDublicates("У меня много много много одинаковых слов")
    

'У меня много одинаковых слов'

## Задание 5
Напишите функцию, которая будет проверять номер сотового телефона на валидность, если он валиден, то переводить его в формат:   
+7(xxx)-xxx-xx-xx  
Постарайтесь предусмотреть как можно больше адекватных форматов изначального ввода номера.
Примеры правильных и неправильных форматов:
- +1 555 555-55-55;
- 5555555555;
- 555555555;
- 8(555)555-55-55;
- +7 555 555 55 55;
- 7(555) 555-55-55;
- +7 555+555+55+55.

In [5]:
def validateRusMobile(txt):
    # Удалим все не-цифры
    nonFormattedTxt = txt
    txt = re.sub(r"[^\d]","",txt)
    # Ожидаем, что телефон идет с начала строки до конца 
    pattern = r"^(?:(?:7|8)*(\d{3})(\d{3})(\d{2})(\d{2}))$"
    
    # Российкий мобильный номер (по шаблону +7*(9xx)-xxx-xx-xx)
    #pattern = r"^(?:(?:7|8)*(9\d{2})(\d{3})(\d{2})(\d{2}))$"   
    
    result = re.findall(pattern,txt)    
    formattedPhoneNum = None
    if len(result) > 0:
        formattedPhoneNum = "+7 ({0})-{1}-{2}-{3}".format(*result[0])
    return (nonFormattedTxt, formattedPhoneNum)

phones = ([
    "+1 555 555-55-55",  # Not Ok - начинается не на +7 / 8
    "5555555555",        # Ok
    "555555555",         # Not Ok - мало цифр
    "8(555)555-55-55",   # Ok
    "+7 555 555 55 55",  # Ok
    "7(555) 555-55-55",  # Ok
    "+7 555+555+55+55.", # Ok
    "+42 555.123.4567",  # Not Ok - начинается не на +7 / 8
    "+1-(800)-123-4567", # Not Ok - начинается не на +7 / 8
    "+7 555 1234567",    # Ok
    "+7(926)1234567",    # Ok
    "(926) 1234567",     # Ok
    "+79261234567",      # Ok
    "926 1234567",       # Ok
    "9261234567",        # Ok
    "1234567",           # Not Ok - мало цифр
    "123-4567",          # Not Ok - мало цифр
    "123-89-01",         # Not Ok - мало цифр
    "495 1234567",       # Ok
    "469 123 45 67",     # Ok
    "89261234567",       # Ok
    "8 (926) 123456",    # Ok
    "926.123.4567",      # Ok
    "415-555-1234",      # Ok
    "650-555-2345",      # Ok
    "(416)555-3456",     # Ok
    "202 555 4567",      # Ok
    "4035555678",        # Ok
    "1 416 555 9292"     # Not ok - начинается не на +7 / 8
          ])

for txt in phones:
    print(validateRusMobile(txt))


('+1 555 555-55-55', None)
('5555555555', '+7 (555)-555-55-55')
('555555555', None)
('8(555)555-55-55', '+7 (555)-555-55-55')
('+7 555 555 55 55', '+7 (555)-555-55-55')
('7(555) 555-55-55', '+7 (555)-555-55-55')
('+7 555+555+55+55.', '+7 (555)-555-55-55')
('+42 555.123.4567', None)
('+1-(800)-123-4567', None)
('+7 555 1234567', '+7 (555)-123-45-67')
('+7(926)1234567', '+7 (926)-123-45-67')
('(926) 1234567', '+7 (926)-123-45-67')
('+79261234567', '+7 (926)-123-45-67')
('926 1234567', '+7 (926)-123-45-67')
('9261234567', '+7 (926)-123-45-67')
('1234567', None)
('123-4567', None)
('123-89-01', None)
('495 1234567', '+7 (495)-123-45-67')
('469 123 45 67', '+7 (469)-123-45-67')
('89261234567', '+7 (926)-123-45-67')
('8 (926) 123456', '+7 (892)-612-34-56')
('926.123.4567', '+7 (926)-123-45-67')
('415-555-1234', '+7 (415)-555-12-34')
('650-555-2345', '+7 (650)-555-23-45')
('(416)555-3456', '+7 (416)-555-34-56')
('202 555 4567', '+7 (202)-555-45-67')
('4035555678', '+7 (403)-555-56-78')
('1 41